<div align="center">
  <span style="color: white; font-size: 45px;">
    Image Classification
  </span>
</div>

## Import Library

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## Define transformations to apply to each image

In [53]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),      # Resize images to 64x64 pixels
    transforms.ToTensor(),            # Convert to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize pixel values to range [-1, 1]
])

## Import dataset

In [67]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
import json
import numpy as np
import os

# Load the nutritional data from the JSON file
with open('D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/nutritional_data.json', 'r') as f:
    nutritional_data = json.load(f)

# Define image size
IMG_SIZE = (150, 150)

# Paths to dataset
train_dir = 'D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train'
test_dir = 'D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/test'

# Create ImageDataGenerators for loading images
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=30, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Generate data for training and testing
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical'
)

# Print class labels detected by the generator
print("Class labels in the training data:")
print(train_generator.class_indices)

# Define the model architecture with a second output for the quantity
def create_model():
    input_layer = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    
    # Convolutional layers
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # Flatten the output from the convolutional layers
    x = Flatten()(x)
    
    # Class prediction output
    class_output = Dense(len(train_generator.class_indices), activation='softmax', name='food_class')(x)
    
    # Quantity prediction output (assuming regression for quantity prediction)
    quantity_output = Dense(1, activation='linear', name='food_quantity')(x)
    
    # Create the final model with multiple outputs
    model = Model(inputs=input_layer, outputs=[class_output, quantity_output])
    
    return model

# Compile the model with two loss functions
model = create_model()
model.compile(optimizer=Adam(), 
              loss={'food_class': 'categorical_crossentropy', 'food_quantity': 'mse'},
              metrics={'food_class': 'accuracy', 'food_quantity': 'mae'})

# Train the model (You can add validation data and epochs as needed)
model.fit(train_generator, epochs=10, validation_data=test_generator)

# Save the trained model
model.save('food_recognition_with_quantity_model.h5')


Found 465 images belonging to 3 classes.


Found 193 images belonging to 3 classes.
Class labels in the training data:
{'ahmok': 0, 'apple': 1, 'orange': 2}
Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 284ms/step - food_class_accuracy: 0.5257 - loss: 0.8228 - val_food_class_accuracy: 0.8135 - val_loss: 0.5990
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 275ms/step - food_class_accuracy: 0.8497 - loss: 0.4533 - val_food_class_accuracy: 0.8290 - val_loss: 0.3950
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 280ms/step - food_class_accuracy: 0.9304 - loss: 0.2005 - val_food_class_accuracy: 0.9326 - val_loss: 0.3292
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 281ms/step - food_class_accuracy: 0.9478 - loss: 0.1662 - val_food_class_accuracy: 0.9067 - val_loss: 0.3450
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 282ms/step - food_class_accuracy: 0.9520 - loss: 0.1459 - val_food_class_accuracy: 0.9119 - val_loss: 0.3106
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 292ms/step - food_class_accuracy: 0.9575 - loss: 0.1258 - val_food_class_accuracy: 0.9171 - val

In [69]:
def predict_image(img_url, nutrition_data):
    img_array = load_image_from_url(img_url, target_size=(150, 150))
    
    # Load the trained model
    model = tf.keras.models.load_model('food_recognition_with_quantity_model.h5')
    
    # Get predictions (food class and quantity)
    food_class_pred, food_quantity_pred = model.predict(img_array)
    
    food_class = np.argmax(food_class_pred, axis=1)
    food_name = list(nutrition_data.keys())[food_class[0]]  # Get the food name based on the class
    
    # Adjust nutrition based on predicted quantity
    quantity = food_quantity_pred[0][0]  # Predicted quantity
    
    food_nutrition = nutrition_data.get(food_name, None)
    
    if food_nutrition:
        adjusted_calories = food_nutrition['calories'] * quantity
        adjusted_fat = food_nutrition['fat'] * quantity
        adjusted_protein = food_nutrition['protein'] * quantity
        adjusted_sugar = food_nutrition['sugar'] * quantity
        
        print(f"Predicted Food: {food_name.capitalize()}")
        print(f"Predicted Quantity: {quantity}")
        print(f"Calories: {adjusted_calories} kcal")
        print(f"Fat: {adjusted_fat} g")
        print(f"Protein: {adjusted_protein} g")
        print(f"Sugar: {adjusted_sugar} g")
    else:
        print("Nutrition data not available for this food.")


## Multiple taks

In [55]:
def build_multi_task_model():
    # Input layer for the images
    image_input = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

    # Convolutional layers for feature extraction
    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = MaxPooling2D(2, 2)(x)
    
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)

    # Flatten the layers
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Output for food classification (categorical classification)
    food_output = Dense(len(nutritional_data), activation='softmax', name='food_classification')(x)

    # Output for nutritional predictions (regression for 4 values: calories, fat, protein, sugar)
    nutrition_output = Dense(4, activation='linear', name='nutrition_prediction')(x)

    # Define the model with two outputs
    model = Model(inputs=image_input, outputs=[food_output, nutrition_output])

    return model


## Model Computing

In [56]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator instance for training and testing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Use target_size=(224, 224) to resize images to the required size
train_generator = train_datagen.flow_from_directory(
    'D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train',  # Path to your training data directory
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=32,
    class_mode='categorical'  # Change as needed depending on your task
)

test_generator = test_datagen.flow_from_directory(
    'D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/test',  # Path to your testing data directory
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=32,
    class_mode='categorical'  # Change as needed depending on your task
)


Found 465 images belonging to 3 classes.
Found 193 images belonging to 3 classes.


## Model Training

In [57]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()  # Enable eager execution if not already enabled


In [58]:
# Training the model for 15 epochs
epochs = 15

# Ensure the model is compiled correctly before fitting
history = model.fit(train_generator,  # Training data generator (ensure this is defined correctly)
    validation_data=test_generator,  # Validation data generator (ensure this is defined correctly)
    epochs=epochs
)

# Save the trained model to a file
model.save('food_recognition_with_nutrition_models.h5')

print("Model saved successfully.")


Epoch 1/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 527ms/step - food_classification_accuracy: 0.9850 - loss: 0.0428 - val_food_classification_accuracy: 0.9223 - val_loss: 0.4822
Epoch 2/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 493ms/step - food_classification_accuracy: 0.9945 - loss: 0.0265 - val_food_classification_accuracy: 0.9326 - val_loss: 0.5292
Epoch 3/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 490ms/step - food_classification_accuracy: 0.9947 - loss: 0.0273 - val_food_classification_accuracy: 0.9171 - val_loss: 0.5954
Epoch 4/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 486ms/step - food_classification_accuracy: 0.9988 - loss: 0.0127 - val_food_classification_accuracy: 0.9223 - val_loss: 0.5787
Epoch 5/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 487ms/step - food_classification_accuracy: 0.9969 - loss: 0.0165 - val_food_classification_accuracy: 0.9171 - val_loss: 0.5715
Epoch 6/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 482ms/step - food_classification_accuracy: 1.0000 - loss: 0.0119 - val_food_classification_accuracy: 0.9171 - val_loss:

Model saved successfully.


In [59]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


## Model Evaluation

In [2]:
import requests
import json
from PIL import Image
import numpy as np
import tensorflow as tf

def load_image_from_url(img_url, target_size=(224, 224)):
    # Fetch the image from the URL
    response = requests.get(img_url, stream=True)
    img = Image.open(response.raw)
    
    # Resize the image to match the model input size
    img = img.resize(target_size)
    
    # Convert the image to a numpy array and normalize it
    img_array = np.array(img) / 255.0  # Normalize if necessary
    
    # Add batch dimension (model expects 4D input: batch_size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

def load_nutrition_info(json_file='D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/nutritional_data.json'):
    with open(json_file, 'r') as file:
        nutrition_data = json.load(file)
    return nutrition_data

def predict_image(img_url, nutrition_data, class_labels):
    # Load and preprocess the image
    img_array = load_image_from_url(img_url, target_size=(224, 224))
    
    # Load the model (ensure you have the trained model saved as .h5)
    model = tf.keras.models.load_model('food_recognition_with_nutrition_models.h5')
    
    # Get the predictions for both outputs (food classification and nutrition prediction)
    food_classification_pred, nutrition_prediction_pred = model.predict(img_array)
    
    # Process the results
    food_class = np.argmax(food_classification_pred, axis=1)  # Get the class with the highest probability
    food_name = list(class_labels.keys())[food_class[0]]  # Get the food name based on the class index
    
    # Get the predicted nutrition value
    food_nutrition = nutrition_data.get(food_name, None)
    
    if food_nutrition:
        print(f"Predicted Food: {food_name.capitalize()}")
        print(f"Calories: {food_nutrition['calories']} kcal")
        print(f"Fat: {food_nutrition['fat']} g")
        print(f"Protein: {food_nutrition['protein']} g")
        print(f"Sugar: {food_nutrition['sugar']} g")
    else:
        print("Nutrition data not available for this food.")

# Example image URL (replace with an actual image URL you want to test)
img_url = 'https://thumbs.dreamstime.com/b/orage-isolated-white-background-d-style-305530202.jpg'

# Load the nutrition data from the JSON file
nutrition_data = load_nutrition_info()

# Define the class labels for mapping prediction to food names
class_labels = {'ahmok': 0, 'apple': 1, 'orange': 2}

# Make predictions for the image
predict_image(img_url, nutrition_data, class_labels)


KeyboardInterrupt: 

In [88]:
import requests
import json
from PIL import Image
import numpy as np
import tensorflow as tf

def load_image_from_url(img_url, target_size=(224, 224)):
    # Fetch the image from the URL
    response = requests.get(img_url, stream=True)
    img = Image.open(response.raw)
    
    # Resize the image to match the model input size
    img = img.resize(target_size)
    
    # Convert the image to a numpy array and normalize it
    img_array = np.array(img) / 255.0  # Normalize if necessary
    
    # Add batch dimension (model expects 4D input: batch_size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

def load_nutrition_info(json_file='D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/nutritional_data.json'):
    with open(json_file, 'r') as file:
        nutrition_data = json.load(file)
    return nutrition_data

def predict_image(img_url, nutrition_data, class_labels):
    # Load and preprocess the image
    img_array = load_image_from_url(img_url, target_size=(224, 224))
    
    # Load the model (ensure you have the trained model saved as .h5)
    model = tf.keras.models.load_model('food_recognition_with_nutrition_models.h5')
    
    # Get the predictions for both outputs (food classification and nutrition prediction)
    food_classification_pred, nutrition_prediction_pred = model.predict(img_array)
    
    # Process the food classification results
    food_class = np.argmax(food_classification_pred, axis=1)  # Get the class with the highest probability
    food_name = list(class_labels.keys())[food_class[0]]  # Get the food name based on the class index
    
    # Get the predicted nutrition value
    food_nutrition = nutrition_data.get(food_name, None)
    
    # Number of food items (For now, assuming one fruit per image; you can integrate object detection for actual counting)
    number_of_food = 3  # Change this if you want to detect multiple items, e.g., with an object detection model
    
    # Calculate total nutrition for the detected number of food items
    if food_nutrition:
        total_calories = food_nutrition['calories'] * number_of_food
        total_fat = food_nutrition['fat'] * number_of_food
        total_protein = food_nutrition['protein'] * number_of_food
        total_sugar = food_nutrition['sugar'] * number_of_food
        
        # Display the results
        print(f"Predicted Food: {food_name.capitalize()}")
        print(f"Predicted Food Class: {food_class[0]} (Index)")
        print(f"Number of Food Items: {number_of_food}")
        print(f"Total Calories: {total_calories} kcal")
        print(f"Total Fat: {total_fat} g")
        print(f"Total Protein: {total_protein} g")
        print(f"Total Sugar: {total_sugar} g")
    else:
        print("Nutrition data not available for this food.")

# Example image URL (replace with an actual image URL you want to test)
img_url = 'https://5.imimg.com/data5/AK/RA/MY-68428614/apple.jpg'

# Load the nutrition data from the JSON file
nutrition_data = load_nutrition_info()

# Define the class labels for mapping prediction to food names
class_labels = {'ahmok': 0, 'apple': 1, 'orange': 2}  # Update this with your actual labels

# Make predictions for the image
predict_image(img_url, nutrition_data, class_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
Predicted Food: Apple
Predicted Food Class: 1 (Index)
Number of Food Items: 3
Total Calories: 285 kcal
Total Fat: 0.8999999999999999 g
Total Protein: 1.5 g
Total Sugar: 57 g


# Parth2

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50

def build_model(num_classes=3):  # Adjust based on your actual number of classes
    # Load a pre-trained ResNet50 model (without the top classification layer)
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the base model layers to retain pre-trained weights
    base_model.trainable = False
    
    # Add custom layers on top for both tasks
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    
    # Task 1: Classification head (food classification)
    class_output = layers.Dense(num_classes, activation='softmax', name='class_output')(x)
    
    # Task 2: Quantity head (predict number of food items)
    quantity_output = layers.Dense(1, activation='linear', name='quantity_output')(x)

    # Create the final model
    model = models.Model(inputs=base_model.input, outputs=[class_output, quantity_output])
    
    model.compile(optimizer='adam',
                  loss={'class_output': 'categorical_crossentropy', 'quantity_output': 'mse'},
                  metrics={'class_output': 'accuracy', 'quantity_output': 'mae'})
    
    return model


In [79]:
import requests
import numpy as np
from PIL import Image

def load_image_from_url(img_url, target_size=(224, 224)):
    # Fetch the image from the URL
    response = requests.get(img_url, stream=True)
    img = Image.open(response.raw)
    
    # Resize the image to match the model input size
    img = img.resize(target_size)
    
    # Convert the image to a numpy array and normalize it
    img_array = np.array(img) / 255.0  # Normalize if necessary
    
    # Add batch dimension (model expects 4D input: batch_size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

def predict_image(img_url, model, class_labels, nutrition_data):
    # Load and preprocess the image
    img_array = load_image_from_url(img_url, target_size=(224, 224))
    
    # Make predictions
    class_pred, quantity_pred = model.predict(img_array)
    
    # Get the predicted class (food type)
    food_class = np.argmax(class_pred, axis=1)
    food_name = list(class_labels.keys())[food_class[0]]
    
    # Get the predicted quantity (how many items)
    number_of_food = round(quantity_pred[0][0])
    
    # Get the nutrition info for the predicted food
    food_nutrition = nutrition_data.get(food_name, None)
    
    # Calculate total nutrition
    if food_nutrition:
        total_calories = food_nutrition['calories'] * number_of_food
        total_fat = food_nutrition['fat'] * number_of_food
        total_protein = food_nutrition['protein'] * number_of_food
        total_sugar = food_nutrition['sugar'] * number_of_food
        
        # Display the results
        print(f"Predicted Food: {food_name.capitalize()}")
        print(f"Predicted Food Class: {food_class[0]} (Index)")
        print(f"Number of Food Items: {number_of_food}")
        print(f"Total Calories: {total_calories} kcal")
        print(f"Total Fat: {total_fat} g")
        print(f"Total Protein: {total_protein} g")
        print(f"Total Sugar: {total_sugar} g")
    else:
        print("Nutrition data not available for this food.")


In [81]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def preprocess_and_augment_data(image_dir, batch_size=32, target_size=(224, 224)):
    # Initialize ImageDataGenerator for data augmentation
    datagen = ImageDataGenerator(
        rescale=1.0/255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Flow images from the directory for training (make sure your data is organized in subdirectories by class)
    train_data = datagen.flow_from_directory(
        image_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    return train_data


In [82]:
# Load your data for training
train_data = preprocess_and_augment_data(image_dir='D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train', batch_size=32)

# Build the model
model = build_model(num_classes=len(class_labels))

# Train the model
model.fit(train_data, epochs=10, steps_per_epoch=len(train_data) // 32)

# Save the trained model
model.save('food_recognition_with_nutrition_modell.h5')


Found 465 images belonging to 3 classes.


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 18s 0us/step
Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - class_output_accuracy: 0.2423 - loss: 1.9905
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - class_output_accuracy: 0.4418 - loss: 0.7671
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - class_output_accuracy: 0.5088 - loss: 0.7183
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - class_output_accuracy: 0.6046 - loss: 0.7595
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - class_output_accuracy: 0.5126 - loss: 0.7455
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - class_output_accuracy: 0.5334 - loss: 0.7170
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - class_output_accuracy: 0.6009 - loss: 0.7287
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - class_output_accuracy: 0.5573 - loss: 0.7096
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - class_output_accuracy: 0.5670 - loss: 0.7078
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - class_output_accuracy:

In [94]:
from tensorflow.keras.metrics import MeanSquaredError

# Load the trained model with custom metrics
def load_trained_model(model_path='food_recognition_with_nutrition_modell.h5'):
    model = tf.keras.models.load_model(model_path, custom_objects={'mse': MeanSquaredError()})
    return model


In [100]:
import tensorflow as tf
from tensorflow.keras.metrics import MeanSquaredError

# Load the trained model (with custom objects, if needed)
def load_trained_model(model_path='food_recognition_with_nutrition_modell.h5'):
    model = tf.keras.models.load_model(model_path, custom_objects={'MeanSquaredError': MeanSquaredError()})
    return model


In [105]:
import numpy as np
import requests
from PIL import Image
import tensorflow as tf
import json  # Make sure to import json for loading the nutrition data

# Load the nutrition data (replace this with the actual path to your nutrition data file)
def load_nutrition_info(json_file='nutritional_data.json'):
    with open(json_file, 'r') as file:
        nutrition_data = json.load(file)
    return nutrition_data

# Load an image from URL and preprocess it
def load_image_from_url(img_url, target_size=(224, 224)):
    # Fetch the image from the URL
    response = requests.get(img_url, stream=True)
    img = Image.open(response.raw)
    
    # Resize the image to match the model input size
    img = img.resize(target_size)
    
    # Convert the image to a numpy array and normalize it
    img_array = np.array(img) / 255.0  # Normalize if necessary
    
    # Add batch dimension (model expects 4D input: batch_size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

# Load the trained model (ensure you have the trained model saved)
def load_trained_model(model_path='food_recognition_with_nutrition_models.h5'):
    model = tf.keras.models.load_model(model_path)
    return model

# Predict the food type, quantity, and total nutrition from an image
def predict_food_and_quantity(img_url, model, class_labels, nutrition_data):
    # Load and preprocess the image
    img_array = load_image_from_url(img_url, target_size=(224, 224))
    
    # Make predictions (food classification and quantity prediction)
    food_class_pred, quantity_pred = model.predict(img_array)
    
    # Get the predicted food class (food type)
    food_class = np.argmax(food_class_pred, axis=1)
    food_name = list(class_labels.keys())[food_class[0]]  # Map index to food name
    
    # Get the predicted quantity (number of food items)
    # If quantity prediction is continuous, round it to the nearest integer
    number_of_food = max(1, round(quantity_pred[0][0]))  # Ensure quantity is at least 1
    
    # Get the nutrition info for the predicted food
    food_nutrition = nutrition_data.get(food_name, None)
    
    # Calculate total nutrition for the predicted quantity
    if food_nutrition:
        total_calories = food_nutrition['calories'] * number_of_food
        total_fat = food_nutrition['fat'] * number_of_food
        total_protein = food_nutrition['protein'] * number_of_food
        total_sugar = food_nutrition['sugar'] * number_of_food
        
        # Display the results
        print(f"Predicted Food: {food_name.capitalize()}")
        print(f"Predicted Food Class: {food_class[0]} (Index)")
        print(f"Number of Food Items: {number_of_food}")
        print(f"Total Calories: {total_calories} kcal")
        print(f"Total Fat: {total_fat} g")
        print(f"Total Protein: {total_protein} g")
        print(f"Total Sugar: {total_sugar} g")
    else:
        print("Nutrition data not available for this food.")

# Example image URL (replace with an actual image URL you want to test)
img_url = 'https://assets.clevelandclinic.org/transform/cd71f4bd-81d4-45d8-a450-74df78e4477a/Apples-184940975-770x533-1_jpg'

# Example class labels (ensure you have the correct mapping)
class_labels = {'ahmok': 0, 'apple': 1, 'orange': 2}  # Update this based on your actual classes

# Load the nutrition data (adjust path to your file)
nutrition_data = load_nutrition_info('nutritional_data.json')

# Load the trained model
model = load_trained_model('food_recognition_with_nutrition_models.h5')

# Predict food type, quantity, and total nutrition
predict_food_and_quantity(img_url, model, class_labels, nutrition_data)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted Food: Apple
Predicted Food Class: 1 (Index)
Number of Food Items: 1
Total Calories: 95 kcal
Total Fat: 0.3 g
Total Protein: 0.5 g
Total Sugar: 19 g


In [107]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def create_train_generator(directory, batch_size=32, target_size=(224, 224)):
    # Initialize the ImageDataGenerator with rescaling
    train_datagen = ImageDataGenerator(rescale=1./255)

    # Get class labels and quantity information from the directory structure
    def quantity_label_from_dir_name(dir_name):
        # You need to define how to extract the quantity (number of food items)
        # For example, if the folder name is "apple_5", extract the quantity '5'
        return int(dir_name.split('_')[-1])  # Assuming the quantity is at the end of the folder name

    # Create a custom generator to yield both image data and (class, quantity) labels
    def custom_train_generator():
        for batch_x, batch_y in train_datagen.flow_from_directory(
            directory,
            target_size=target_size,
            batch_size=batch_size,
            class_mode='categorical'  # Only food class
        ):
            # Get the directory names for the labels
            labels = batch_y
            quantity_labels = [quantity_label_from_dir_name(name) for name in os.listdir(directory)]
            
            # Yield images, food class labels, and quantity labels
            yield batch_x, {'food_class': labels, 'quantity': quantity_labels}

    return custom_train_generator()



In [110]:
import re

def quantity_label_from_dir_name(dir_name):
    # Use regex to search for a number at the end of the directory name
    match = re.search(r'(\d+)$', dir_name)  # This will match any digits at the end of the string
    
    if match:
        return int(match.group(1))  # Extract and return the number as an integer
    else:
        # If no number is found, return a default value or handle the case accordingly
        print(f"Warning: No quantity found in directory name: {dir_name}")
        return 1  # Default value, adjust as needed


In [109]:
import json

# Load the nutritional data from the JSON file
def load_nutrition_info(json_file='nutritional_data.json'):
    with open(json_file, 'r') as f:
        nutrition_data = json.load(f)
    return nutrition_data

# Extract nutrition for a food item
def get_food_nutrition(food_name, nutrition_data):
    # Use the food name to fetch nutritional information
    food_data = nutrition_data.get(food_name)
    if food_data:
        return food_data
    else:
        print(f"Warning: Nutrition data for {food_name} not found!")
        return None  # Return None or a default value if the food is not found

# Example usage
nutrition_data = load_nutrition_info('nutritional_data.json')
food_name = 'apple'  # Example food
nutrition_info = get_food_nutrition(food_name, nutrition_data)

if nutrition_info:
    print(f"Nutrition data for {food_name}: {nutrition_info}")


Nutrition data for apple: {'calories': 95, 'fat': 0.3, 'protein': 0.5, 'sugar': 19}


In [111]:
# Set the number of food classes (e.g., 3 classes: apple, orange, banana)
num_classes = 3

# Build the model
model = build_model(num_classes=num_classes)

# Create the training generator
train_generator = create_train_generator('D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train')

# Train the model
model.fit(train_generator, epochs=10, steps_per_epoch=100)  # Adjust the steps per epoch based on your dataset size


Found 465 images belonging to 3 classes.


ValueError: invalid literal for int() with base 10: 'ahmok'

In [ ]:
import torch
from PIL import Image
import numpy as np

# Load the YOLOv5 model (using PyTorch)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Use 'yolov5m' or 'yolov5l' for larger models

# Load image from URL or local path
img_url = 'https://grantourismotravels.com/wp-content/uploads/2017/05/Authentic-Fish-Amok-Recipe-Steamed-Fish-Curry-Cambodia-Copyright-2022-Terence-Carter-Grantourismo-T-500x375.jpg'
img = Image.open(requests.get(img_url, stream=True).raw)

# Run inference (detect objects in the image)
results = model(img)

# Results contains detected objects with labels, confidence, and bounding boxes
predicted_labels = results.names  # The class names
predicted_boxes = results.xywh[0]  # Predicted bounding boxes (x, y, width, height)
confidences = results.xywh[0][:, 4]  # Confidence scores
classes = results.xywh[0][:, 5].tolist()  # Class indices

# Count instances of each class (food items)
food_counts = {}
for cls in classes:
    food_name = predicted_labels[int(cls)]
    food_counts[food_name] = food_counts.get(food_name, 0) + 1

# Nutrition data for the food items (example data structure)
nutrition_data = {
    'apple': {'calories': 100, 'fat': 0.5, 'protein': 0.5, 'sugar': 19},
    'orange': {'calories': 80, 'fat': 0.3, 'protein': 1.5, 'sugar': 18},
    'grape': {'calories': 80, 'fat': 0.3, 'protein': 1.5, 'sugar': 18}
    
    
}

# Calculate total nutrition based on the detected quantity
for food_name, count in food_counts.items():
    if food_name in nutrition_data:
        food_nutrition = nutrition_data[food_name]
        total_calories = food_nutrition['calories'] * count
        total_fat = food_nutrition['fat'] * count
        total_protein = food_nutrition['protein'] * count
        total_sugar = food_nutrition['sugar'] * count

        # Display results
        print(f"Predicted Food: {food_name.capitalize()}")
        print(f"Number of Food Items: {count}")
        print(f"Total Calories: {total_calories} kcal")
        print(f"Total Fat: {total_fat} g")
        print(f"Total Protein: {total_protein} g")
        print(f"Total Sugar: {total_sugar} g")
    else:
        print(f"Nutrition data not available for {food_name}.")


Using cache found in C:\Users\ADMIN/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-2-12 Python-3.11.5 torch-2.2.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Nutrition data not available for carrot.


In [1]:
import numpy as np
import requests
from PIL import Image
import torch  # We need torch to load YOLOv5
import json  # To load the nutrition data

# Load the nutrition data (replace this with the actual path to your nutrition data file)
def load_nutrition_info(json_file='nutritional_data.json'):
    with open(json_file, 'r') as file:
        nutrition_data = json.load(file)
    return nutrition_data

# Load an image from URL and preprocess it
def load_image_from_url(img_url):
    # Fetch the image from the URL
    response = requests.get(img_url, stream=True)
    img = Image.open(response.raw)
    
    return img

# Load YOLOv5 model (use a custom-trained YOLOv5 model if you have one)
def load_yolov5_model(model_path='yolov5s.pt'):  # yolov5s is a small, pre-trained model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)  # Load custom model
    return model

# Predict the food type, quantity, and total nutrition from an image using YOLOv5
def predict_food_and_quantity(img_url, model, nutrition_data, class_labels):
    # Load and preprocess the image
    img = load_image_from_url(img_url)
    
    # Perform inference with YOLOv5
    results = model(img)  # Inference
    
    # Get the results from YOLO (bounding boxes, labels, and confidence scores)
    detected_classes = results.names  # Class names predicted by YOLO
    detected_boxes = results.xywh[0]  # Bounding boxes (x, y, width, height)
    confidences = results.xywh[0][:, 4]  # Confidence scores for each detection
    class_indices = results.xywh[0][:, 5].tolist()  # Class indices for each detected object
    
    # Count and calculate nutrition for each detected food item
    food_count = {}
    
    for idx, cls_idx in enumerate(class_indices):
        food_name = detected_classes[cls_idx]
        
        # Increase the count for each detected food type
        if food_name in food_count:
            food_count[food_name] += 1
        else:
            food_count[food_name] = 1
    
    # Calculate nutrition for each food detected
    for food_name, count in food_count.items():
        food_nutrition = nutrition_data.get(food_name, None)
        
        if food_nutrition:
            total_calories = food_nutrition['calories'] * count
            total_fat = food_nutrition['fat'] * count
            total_protein = food_nutrition['protein'] * count
            total_sugar = food_nutrition['sugar'] * count
            
            # Display the results
            print(f"Predicted Food: {food_name.capitalize()}")
            print(f"Count: {count}")
            print(f"Total Calories: {total_calories} kcal")
            print(f"Total Fat: {total_fat} g")
            print(f"Total Protein: {total_protein} g")
            print(f"Total Sugar: {total_sugar} g")
        else:
            print(f"Nutrition data not available for {food_name}.")

# Example image URL (replace with an actual image URL you want to test)
img_url = 'https://ca-times.brightspotcdn.com/dims4/default/b243748/2147483647/strip/true/crop/8100x5400+0+0/resize/2000x1333!/quality/75/?url=https%3A%2F%2Fcalifornia-times-brightspot.s3.amazonaws.com%2F1e%2Fde%2F897ca4cd4327a0e7d2328bae202c%2F1419161-fo-sophys-review-rad-003.jpg'

# Example class labels (ensure you have the correct mapping for your food classes)
class_labels = {'ahmok': 0, 'apple': 1, 'orange': 2}  # Update this based on your actual classes

# Load the nutrition data (adjust path to your file)
nutrition_data = load_nutrition_info('nutritional_data.json')

# Load the YOLOv5 model
model = load_yolov5_model('yolov5s.pt')  # Replace with your custom-trained model if available

# Predict food type, quantity, and total nutrition
predict_food_and_quantity(img_url, model, nutrition_data, class_labels)


Using cache found in C:\Users\ADMIN/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-2-12 Python-3.11.5 torch-2.2.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Nutrition data not available for sandwich.


In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import tensorflow as tf
from PIL import Image
import numpy as np
import io

# Load your trained model
model = tf.keras.models.load_model('food_recognition_with_nutrition_models.h5')

app = FastAPI()

# Function to preprocess the image and make predictions
def process_image(image: UploadFile):
    # Read image
    img_bytes = image.file.read()
    img = Image.open(io.BytesIO(img_bytes))

    # Preprocess the image as required by your model
    img = img.resize((224, 224))  # Resize to the expected input size for the model
    img_array = np.array(img) / 255.0  # Normalize the image

    # Add batch dimension to image array
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    prediction = model.predict(img_array)

    # Assuming the model returns nutrition values directly
    nutrition_data = {
        "calories": prediction[0][0],
        "protein": prediction[0][1],
        "fat": prediction[0][2],
        "sugar": prediction[0][3],
    }
    return nutrition_data

@app.post("/upload/")
async def upload_image(file: UploadFile = File(...)):
    try:
        nutrition_data = process_image(file)
        return JSONResponse(content=nutrition_data)
    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)


In [8]:
import tensorflow as tf

# Load your Keras model
model = tf.keras.models.load_model('C:/xampp/htdocs/php-practics/fnapp/model/food_recognition_with_nutrition_models.h5')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('C:/xampp/htdocs/php-practics/fnapp/model/fn.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmpb7iix0k5\assets


INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmpb7iix0k5\assets


Saved artifact at 'C:\Users\ADMIN\AppData\Local\Temp\tmpb7iix0k5'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_6')
Output Type:
  List[TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)]
Captures:
  1609766711568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1609766710224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1609766711760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1609766710992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1609766710800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1609766704272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1609766710032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1609766709264: TensorSpec(shape=(), dtype=tf.resource, name=None)
